In [18]:
from langchain.document_loaders import PyPDFDirectoryLoader
from sentence_transformers import SentenceTransformer, util
import re
import pickle

In [19]:
def picklr(file, ob):
    with open(file, 'wb') as f:
        pickle.dump(ob, f)

def unpicklr(file):
    with open(file, 'rb') as f:
        ob = pickle.load(f)
    return ob

In [9]:
loader = PyPDFDirectoryLoader("fichas/")

In [11]:
docs = loader.load()

In [12]:
my_dict = {}
for doc in docs:
    try:
        my_dict[doc.metadata["source"]] += doc.page_content
    except:
        my_dict[doc.metadata["source"]] = doc.page_content

In [13]:
SENTENCE_TRANSFORMER = SentenceTransformer("intfloat/multilingual-e5-large")

In [14]:
secciones = ["REQUISITOS (ASIGNATURAS O MATERIAS CUYO CONOCIMIENTO SE PRESUPONE)",
             "OBJETIVOS",
             "DESCRIPCIÓN DE CONTENIDOS: PROGRAMA",
             "ACTIVIDADES FORMATIVAS, METODOLOGÍA A UTILIZAR Y RÉGIMEN DE TUTORÍAS",
             "SISTEMA DE EVALUACIÓN",
             "BIBLIOGRAFÍA BÁSICA"]

pag_regex = re.compile(r'Página \d+ de \d+')
corpus = []
vecs = []

for _, texto in my_dict.items():
    title = texto[: texto.find("Departamento")].upper()

    for sec, next_sec in zip(secciones, secciones[1:] + [None]):
        start_index = texto.find(sec)
        end_index = texto.find(next_sec) if next_sec is not None else None
        
        if start_index != -1:
            contenido = texto[start_index + len(sec):end_index]
            chunk = title + sec + contenido
            chunk = chunk.replace("\n", " ")
            chunk = re.sub(pag_regex, "", chunk)
            vec = SENTENCE_TRANSFORMER.encode(chunk)
            corpus.append(chunk)
            vecs.append(vec)



In [25]:
picklr("./ingested_data/corpus.pkl", corpus)
picklr("./ingested_data/vecs.pkl", vecs)

In [26]:
corpus = unpicklr("./ingested_data/corpus.pkl")
vecs = unpicklr("./ingested_data/vecs.pkl")

In [28]:
import requests

questions = ["como se evalua redes neuronales?",
             "que libros tengo que comprar para evolutiva",
             "cuanto cuenta el final de etica",
             "que requisitos tiene multiagente",
             "how much is the robotics final exam worth?"]
TOP_SCORES = 1

for question in questions:
    target_vector = SENTENCE_TRANSFORMER.encode(question)
    cosine_similarities = util.pytorch_cos_sim(target_vector, vecs)[0]
    document_similarity_pairs = list(zip(corpus, cosine_similarities.tolist()))
    sorted_pairs = sorted(document_similarity_pairs, key=lambda x: x[1], reverse=True)
    similar_docs = sorted_pairs[:TOP_SCORES]

    for i, (title, score) in enumerate(similar_docs, start=1):
        print(f"Pregunta: {question}\nSimilarity Score: {score:.4f}\n{title}\n")
        data = {
            "model": "llama2",
            "prompt": f"Eres el asistente virtual del máster de Inteligencia Artificial Aplicada en la UC3M. Los estudiantes pueden hacerte preguntas sobre las asignaturas, y tú responderas con la información que te proporciono como contexto. Sé extremadamente breve. Sólo hablas español. Contexto: {title} Pregunta: {question} Respuesta: ",
            "stream": False
        }

        response = requests.post("http://localhost:11434/api/generate", json=data)
        print(response.json()["response"], "\n")

Pregunta: como se evalua redes neuronales?
Similarity Score: 0.8720
REDES DE NEURONAS SISTEMA DE EVALUACIÓN La evaluación continua consistirá en exámenes parciales, realización de prácticas y proyectos de programación y presentación de trabajos. SE 2 Trabajos individuales o en grupo realizados durante el curso La evaluación extraordinaria consistirá en un examen final. SE 3 Examen final Peso porcentual del Examen Final: 0 Peso porcentual del resto de la evaluación: 100 


Las redes neuronales son evaluadas mediante un examen final de 0% y el resto de la evaluación es de 100%. 

Pregunta: que libros tengo que comprar para evolutiva
Similarity Score: 0.8443
COMPUTACIÓN EVOLUTIVA BIBLIOGRAFÍA BÁSICA  - D. Borrajo, J. Gonzalez, P. Isasi Aprendizaje Automático, Sanz y Torres, 2013  - D. Floreano, C. Mattiussi Bio-Inspired Artificial Intelligence: Theories, Methods, and Technologies, MIT Press, 2008  - E. Talbi Metaheuristics: From Design to Implementation, Wiley, 2009 

 ¡Hola! Como asisten

In [ ]:
def query(question, history):    
    target_vector = SENTENCE_TRANSFORMER.encode(question)
    cosine_similarities = util.pytorch_cos_sim(target_vector, vecs)[0]
    document_similarity_pairs = list(zip(corpus, cosine_similarities.tolist()))
    sorted_pairs = sorted(document_similarity_pairs, key=lambda x: x[1], reverse=True)
    similar_docs = sorted_pairs[:TOP_SCORES]

    for i, (title, score) in enumerate(similar_docs, start=1):
        print(f"Pregunta: {question}\nSimilarity Score: {score:.4f}\n{title}\n")
        data = {
            "model": "llama2",
            "prompt": f"Eres el asistente virtual del máster de Inteligencia Artificial Aplicada en la UC3M. Los estudiantes pueden hacerte preguntas sobre las asignaturas, y tú responderas con la información que te proporciono como contexto. Sé extremadamente breve. Sólo hablas español. Contexto: {title} Pregunta: {question} Respuesta: ",
            "stream": False
        }

        response = requests.post("http://localhost:11434/api/generate", json=data)
        print(response.json()["response"], "\n")
        return response.json()["response"]

In [ ]:
import streamlit as st

st.title("Echo Bot")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("What is up?"):
    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    response = f"Echo: {prompt}"
    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        st.markdown(response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

2023-12-28 18:18:50.719 
  command:

    streamlit run /Users/maialenespilanda/Desktop/UC3M/NLP/genai/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2023-12-28 18:18:50.720 Session state does not function when running a script without `streamlit run`


AttributeError: st.session_state has no attribute "messages". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization